In [1]:
import requests
import json
import aiohttp
import asyncio
import json

In [2]:
PATCH = "14.22.1"
all_data = {}
patch_champions_id = []

base_data = f"https://ddragon.leagueoflegends.com/cdn/{PATCH}/data/en_US/champion.json"

request = requests.get(base_data)
data = request.json()

for champion in data["data"].values():
    patch_champions_id.append(champion.get("key"))


In [10]:
async def download_image_splash(session, url, champion_name):
    async with session.get(url) as response:
        if response.status == 200:
            content = await response.read()
            champion_name = champion_name.replace("'", "").replace(" ", "").strip().lower().capitalize()
            with open(f"champions-splash/{champion_name}_splash_centered.jpg", "wb") as f:
                f.write(content)

async def download_image_square(session, url, champion_name):
    async with session.get(url) as response:
        if response.status == 200:
            content = await response.read()
            champion_name = champion_name.replace("'", "").replace(" ", "").strip().lower().capitalize()
            with open(f"champions-square/{champion_name}_square.jpg", "wb") as f:
                f.write(content)

async def fetch_champion_data(session, champion_id):
    async with session.get(f"https://cdn.communitydragon.org/{PATCH}/champion/{str(champion_id)}/data") as response:
        data = await response.json()
        champ_data = {
            "name": data["name"].replace("'", "").replace(" ", "").strip().lower().capitalize(),
            "title": data["title"],
            "tacticalInfo": data["tacticalInfo"],
            "playstyleInfo": data["playstyleInfo"],
            "roles": data["roles"],
        }
        all_data[data["id"]] = champ_data
        image_splash_url = f"https://cdn.communitydragon.org/{PATCH}/champion/{str(champion_id)}/splash-art/centered"
        image_square_url = f"https://cdn.communitydragon.org/{PATCH}/champion/{str(champion_id)}/square"
        
        await download_image_splash(session, image_splash_url, data["name"])
        await download_image_square(session, image_square_url, data["name"])

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for c in patch_champions_id:
            tasks.append(fetch_champion_data(session, c))
        await asyncio.gather(*tasks)

await main()

with open("champions_data.json", "w") as f:
    json.dump(all_data, f, indent=4)

In [6]:
import os

# count images in 'champions-splash' folder
image_folder = ['champions-splash', 'champions-square']

for folder in image_folder:
    image_count = len([name for name in os.listdir(folder) if os.path.isfile(os.path.join(folder, name))])
    print(f"Number of images in {folder}: {image_count}")


Number of images in champions-splash: 169
Number of images in champions-square: 169
